### This notebook will load in, clean, and wrangle two datasets to prepare for NLP and modeling 

In [1]:
# importing standard libraries 
import pandas as pd

In [2]:
# two seperate pre-made datasets to concatinate after cleaning for NLP and modeling
first = pd.read_csv('news.csv')
second = pd.read_csv('train.csv')

In [3]:
first.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


- At first glance:
  - change the label to 1s and 0s 
  - join the title and text 

In [4]:
second.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


- label of 1 is unreliabe/fake
- author is not needed but like with the first df, title and text will be merged
---

In [5]:
print(first.shape)
print(second.shape)

(6335, 4)
(20800, 5)


- Total observations should be 27135

In [6]:
first['text'] = first['title'] + ' ' + first['text']      # joining title and text 
second['text'] = second['title'] + ' ' + second['text']   
first = first[['text', 'label']]                          # only using text and label
second = second[['text', 'label']]
first['label'] = first['label'].apply(lambda x: 1 if x == 'FAKE' else 0) # changing categorical variables to numeric

In [7]:
# concatinating first and second dfs with text and label. 
frames = [first, second] 
main = pd.concat(frames, ignore_index=True)

In [8]:
main

,text,label
0,You Can Smell Hillary’s Fear Daniel Greenfield...,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,1
2,Kerry to go to Paris in gesture of sympathy U....,0
3,Bernie supporters on Twitter erupt in anger ag...,1
4,The Battle of New York: Why This Primary Matte...,0
...,...,...
27130,Rapper T.I.: Trump a ’Poster Child For White S...,0
27131,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",0
27132,Macy’s Is Said to Receive Takeover Approach by...,0
27133,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [9]:
main.isna().sum()

text     597
label      0
dtype: int64

- Only 2% of data has null values and so dropping them seems best 

In [10]:
main.dropna(inplace=True)

In [11]:
main.reset_index(inplace=True, drop=True)

In [12]:
main.isna().sum()

text     0
label    0
dtype: int64

In [13]:
main

,text,label
0,You Can Smell Hillary’s Fear Daniel Greenfield...,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,1
2,Kerry to go to Paris in gesture of sympathy U....,0
3,Bernie supporters on Twitter erupt in anger ag...,1
4,The Battle of New York: Why This Primary Matte...,0
...,...,...
26533,Rapper T.I.: Trump a ’Poster Child For White S...,0
26534,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",0
26535,Macy’s Is Said to Receive Takeover Approach by...,0
26536,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [14]:
# This will be the working dataset that will be cleaned and tokenized
main.to_csv('main1.csv')

---


### This section of the notebook will create tokenized text, lemmatized text, and stemmed text for the modeling process

#### Tokenized with stop words removed, no lemma or stemming yet 

In [20]:
import nltk

In [16]:
# Importing stop words and punctuations to remove from text 
import string
from nltk.corpus import stopwords

stopwords = stopwords.words('english')
stopwords += list(string.punctuation)
stopwords += ['’', '“', '”', '–', "'s", "''", '—', '``', '‘', '``']

In [17]:
# A function to downcase, tokenize, and remove stopwords
def process_article(article):
    article = article.lower()
    tokens = nltk.word_tokenize(article)
    stopwords_removed = [token.lower() for token in tokens if token.lower() not in stopwords]
    return ' '.join(stopwords_removed)

In [18]:
# To keep main df, I'll use a working df to apply the function
working_text = main['text']

In [21]:
# Using tqdm to track progress
from tqdm import tqdm
tqdm.pandas()
working_text = working_text.progress_apply(process_article)

100%|██████████| 26538/26538 [05:55<00:00, 74.75it/s] 


In [23]:
working_text

0        smell hillary fear daniel greenfield shillman ...
1        watch exact moment paul ryan committed politic...
2        kerry go paris gesture sympathy u.s. secretary...
3        bernie supporters twitter erupt anger dnc 'we ...
4        battle new york primary matters primary day ne...
                               ...                        
26533    rapper t.i trump poster child white supremacy ...
26534    n.f.l playoffs schedule matchups odds new york...
26535    macy said receive takeover approach hudson bay...
26536    nato russia hold parallel exercises balkans na...
26537    keeps f-35 alive david swanson author activist...
Name: text, Length: 26538, dtype: object

In [24]:
working_text.to_csv('working_text.csv')


### Lemma 

In [25]:
from nltk.stem import WordNetLemmatizer 

In [26]:
lemma = WordNetLemmatizer()

In [27]:
def lemmafunc(article):
    word_list = nltk.word_tokenize(article)
    lemmatized_output = ' '.join([lemma.lemmatize(w) for w in word_list])
    return lemmatized_output

In [29]:
lemmaText = working_text.progress_apply(lemmafunc)

100%|██████████| 26538/26538 [03:01<00:00, 146.52it/s]


In [30]:
lemmaText

0        smell hillary fear daniel greenfield shillman ...
1        watch exact moment paul ryan committed politic...
2        kerry go paris gesture sympathy u.s. secretary...
3        bernie supporter twitter erupt anger dnc 'we t...
4        battle new york primary matter primary day new...
                               ...                        
26533    rapper t.i trump poster child white supremacy ...
26534    n.f.l playoff schedule matchup odds new york t...
26535    macy said receive takeover approach hudson bay...
26536    nato russia hold parallel exercise balkan nato...
26537    keep f-35 alive david swanson author activist ...
Name: text, Length: 26538, dtype: object

In [31]:
lemmaText.to_csv('lemma_text.csv')

#### Stemming

In [32]:
from nltk.stem import PorterStemmer
portstem = PorterStemmer()

In [33]:
def stemmer(article):
    word_list = nltk.word_tokenize(article)
    stem = ' '.join([portstem.stem(w) for w in word_list])
    return stem

In [34]:
stemText = working_text.progress_apply(stemmer)

100%|██████████| 26538/26538 [08:57<00:00, 49.36it/s]


In [35]:
stemText

0        smell hillari fear daniel greenfield shillman ...
1        watch exact moment paul ryan commit polit suic...
2        kerri go pari gestur sympathi u.s. secretari s...
3        berni support twitter erupt anger dnc 'we tri ...
4        battl new york primari matter primari day new ...
                               ...                        
26533    rapper t.i trump poster child white supremaci ...
26534    n.f.l playoff schedul matchup odd new york tim...
26535    maci said receiv takeov approach hudson bay ne...
26536    nato russia hold parallel exercis balkan nato ...
26537    keep f-35 aliv david swanson author activist j...
Name: text, Length: 26538, dtype: object

In [36]:
stemText.to_csv('stem_text.csv')